<a href="https://colab.research.google.com/github/kyledinh/gpt-prive/blob/main/notebookds/client-to-hf-inference-endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%pip install -U huggingface_hub python-dotenv


[notice] A new release of pip available: 22.2.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from huggingface_hub import InferenceClient

# Load .env variables
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

HF_ACCESS_TOKEN = os.getenv("HF_ACCESS_TOKEN", "add-here-if-not-set-in-env-file")
HF_ENDPOINT_URL = os.getenv("HF_ENDPOINT_URL", "add-here-if-not-set-in-env-file")
 
assert os.getenv("HF_ACCESS_TOKEN", "").startswith("hf_"), "This doesn't look like a valid Hugging Face Token"
print("HF_ACCESS_TOKEN configured")
assert os.getenv("HF_ENDPOINT_URL", "").startswith("https"), "This doesn't look like a valid Hugging Face Endpoint URL"
print("HF_ACCESS_URL configured")

# Streaming Client
client = InferenceClient(HF_ENDPOINT_URL, token=HF_ACCESS_TOKEN)

# generation parameter
gen_kwargs = dict(
    max_new_tokens=512,
    top_k=30,
    top_p=0.9,
    temperature=0.2,
    repetition_penalty=1.02,
    stop_sequences=["\nUser:", "<|endoftext|>", "</s>"],
)

HF_ACCESS_TOKEN configured
HF_ACCESS_URL configured


Non-streaming Example

In [11]:
# prompt
prompt = "What are good motorcycle riding roads in the United States? Give me 4 examples."
response = client.text_generation(prompt, **gen_kwargs)
print(response)


1. Route 66 - stretching from Chicago to Santa Monica, California, this iconic road offers scenic vistas and diverse landscapes.

2. The Blue Ridge Parkway - a 2,000-mile stretch of road that runs through the Appalachian Mountains, providing breathtaking views and winding roads.

3. Highway 1 - also known as the Pacific Coast Highway, this coastal road offers stunning vistas of the Pacific Ocean and rugged coastline.

4. The Great River Road - a 2,300-mile stretch of road that follows the Mississippi River, providing beautiful landscapes, small towns, and historical sites.


Prompt example with Streaming

In [12]:
prompt = "What are good motorcycle riding roads in the United States? Give me 4 examples."

stream = client.text_generation(prompt, stream=True, details=True, **gen_kwargs)

# yield each generated token
for r in stream:
    # skip special tokens
    if r.token.special:
        continue
    # stop if we encounter a stop sequence
    if r.token.text in gen_kwargs["stop_sequences"]:
        break
    # yield the generated token
    print(r.token.text, end = "")
    # yield r.token.text


1. Route 66 (Chicago to Santa Monica, California) 2. The Pacific Coast Highway (San Francisco to Los Angeles) 3. The Blue Ridge Parkway (Virginia to North Carolina) 4. The Great Ocean Road (Victoria, Australia to Adelaide, South Australia)